In [ ]:
!pip install rdkit-pypi -qqq
from rdkit import Chem

from google.colab import files
uploaded = files.upload()
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['gandata.csv']))
# Dataset is now stored in a Pandas Dataframe

import os
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
smiles_train, smiles_test = train_test_split(data["canonical_smiles"], random_state=42)
print (smiles_train.shape)
print (smiles_test.shape)

charset = set("".join(list(data.canonical_smiles))+"!E")
char_to_int = dict((c,i) for i,c in enumerate(charset))
int_to_char = dict((i,c) for i,c in enumerate(charset))
embed = max([len(smile) for smile in data.canonical_smiles]) + 5
print (str(charset))
n_vocab=len(charset)

def vectorize(smiles, embed, n_vocab):
    one_hot = np.zeros((smiles.shape[0],embed, n_vocab), dtype=np.int8)
    for i, smile in enumerate(smiles):
        # encode the startchar
        one_hot[i, 0, char_to_int["!"]] = 1
        # encode the rest of the chars
        for j, c in enumerate(smile):
            one_hot[i, j + 1, char_to_int[c]] = 1
        # Encode endchar
        one_hot[i, len(smile) + 1:, char_to_int["E"]] = 1
    # Return two, one for input and the other for output
    return one_hot[:, 0:-1, :], one_hot[:, 1:, :]

X_train, Y_train = vectorize(smiles_train.values,embed,n_vocab)
X_test, Y_test = vectorize(smiles_test.values,embed,n_vocab)
print(smiles_train.iloc[0])

print (X_train.shape)
q="".join([int_to_char[idx] for idx in np.argmax(X_train[0,:,:], axis=1)])
print(q)
from keras.models import Model
from keras.layers import Input
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Concatenate
from keras import regularizers

def lstm_model(X,y):
    enc_input=Input(shape=(X.shape[1:]))
    _,state_h,state_c=LSTM(256,return_state=True)(enc_input)
    states= Concatenate(axis=-1)([state_h,state_c])
    bottle_neck= Dense(128, activation='relu')(states)

    unroll = False
    state_h_decoded= Dense(256, activation='relu')(bottle_neck)
    state_c_decoded= Dense(256, activation='relu')(bottle_neck)
    encoder_states= [state_h_decoded, state_c_decoded]
    dec_input= Input(shape=(X.shape[1:]))
    dec1= LSTM(256, return_sequences=True,unroll=unroll)
    decoder_outputs = dec1(dec_input, initial_state=encoder_states)
    decoder_dense= Dense (y.shape[2], activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model= Model(inputs=[enc_input, dec_input], outputs=decoder_outputs)
    return model

model = lstm_model(X_train, Y_train)
model.summary()

from keras.callbacks import History, ReduceLROnPlateau
h = History()
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,patience=10, min_lr=0.000001, verbose=1, min_delta=1e-5)

from tensorflow.keras.optimizers import RMSprop, Adam
opt=Adam(learning_rate=0.005) #Default 0.001
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([X_train,X_train],Y_train,epochs=200,batch_size=1024,shuffle=True,callbacks=[h, rlr],validation_data=([X_test,X_test],Y_test ))
mod_file = 'LSTM_model.h5'
model.save(mod_file)
print("Model Saved")
files.download(mod_file)

encoder_model=Model(inputs=model.layers[0].input, outputs=model.layers[3].output)
encoder_model.summary()

latent_input = Input(shape=(128,))
#reuse_layers
state_h =  model.layers[5](latent_input)
state_c =  model.layers[6](latent_input)
latent_to_states_model = Model(latent_input, [state_h, state_c])
latent_to_states_model.summary()


#Last one is special, we need to change it to stateful, and change the input shape
decoder_inputs = Input(batch_shape=(1, 1, n_vocab))
decoder_lstm = LSTM(256,
                    return_sequences=True,
                    stateful=True
                   )(decoder_inputs)
decoder_outputs = Dense(n_vocab, activation='softmax')(decoder_lstm)
gen_model = Model(decoder_inputs, decoder_outputs)

#Transfer Weights
for i in range(1,3):
    gen_model.layers[i].set_weights(model.layers[i+6].get_weights())
gen_model.save("Blog_simple_samplemodel.h5")
gen_model.summary()

def sample_with_temp(preds, sampling_temp):
    streched= np.log(preds)/ sampling_temp
    streched_probs=np.exp(streched)/np.sum(np.exp(streched))
    return np.random.choice(range(len(streched)),p=streched_probs )

def sample_smiles(latent,sampling_temp):
    #decode states and set Reset the LSTM cells with them
    states = latent_to_states_model.predict(latent)
    gen_model.layers[1].reset_states(states=[states[0],states[1]])
    #Prepare the input char
    startidx = char_to_int["!"]
    samplevec = np.zeros((1,1,n_vocab))
    samplevec[0,0,startidx] = 1
    sequence = ""
    #Loop and predict next char
    for i in range(30):
        preds = gen_model.predict(samplevec)[0][-1]
        if sampling_temp==1.0:
            
            sampleidx = np.argmax(preds)
        else:
            sampleidx=sample_with_temp(preds, sampling_temp)
        samplechar = int_to_char[sampleidx]
        if samplechar != "E":
            sequence += samplechar
            samplevec = np.zeros((1,1,n_vocab))
            samplevec[0,0,sampleidx] = 1
        else:
            break
    return sequence

#from google.colab import files
#uploaded = files.upload()
import pandas as pd
import io
#data1 = pd.read_csv(io.BytesIO(uploaded['bioactivity_data.csv']))
#smiles_t= data1["canonical_smiles"]
#charset = set("".join(list(data1.canonical_smiles))+"!E")
#char_to_int = dict((c,i) for i,c in enumerate(charset))
#int_to_char = dict((i,c) for i,c in enumerate(charset))
#embed = max([len(smile) for smile in data1.canonical_smiles]) + 5
#print (str(charset))
#n_vocab1=len(charset)
#print(n_vocab1)
#X_t, Y_t = vectorize(smiles_t.values,embed,n_vocab1)

test_latent_space=encoder_model.predict(X_test)
print(f'test smiles {test_latent_space.shape}')

sampling_temp= 1.0
test_smile= sample_smiles(test_latent_space[0:1], sampling_temp)
print(test_smile)
print(smiles_test.iloc[0])

sampling_temp= 1.25
test_smile= sample_smiles(test_latent_space[0:1], sampling_temp)
print(test_smile)
print(smiles_test.iloc[0])

from rdkit import Chem
#from rdkit.Chem import Draw, Descriptors
sampling_temp=1.0
wrong = 0
for i in range(100):
    smiles = sample_smiles(test_latent_space[i:i+1],sampling_temp)
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        pass
    else:
        
        wrong = wrong + 1
print ("%0.1F percent wrongly formatted smiles"%(wrong/float(100)*100))

def generate(latent_seed,sampling_temp,scale,quant):
  samples,mols=[],[]
  for i in range(quant):
    latent_vec=latent_seed+scale*(np.random.randn(latent_seed.shape[1]))
    out= sample_smiles(latent_vec,sampling_temp)
    mol= Chem.MolFromSmiles(out)
    if mol:
      mols.append(mol)
      samples.append(out)
  return mols, samples

gen_mols, gen_smiles=[],[]
for i in range(test_latent_space.shape[0]-1):
  latent_seed=  test_latent_space[i:i+1]
  sampling_temp=np.random.uniform(0.75,1.26)
  scale=0.75
  quantity=2
  mols, smiles= generate(latent_seed, sampling_temp,scale,quantity)
  gen_mols.extend(mols)
  gen_smiles.extend(smiles)
  mols,smiles=[],[]
print(gen_smiles)

from google.colab import files
s = "\n"

str1 = s.join(gen_smiles)
with open('example.txt', 'w') as f:
  f.write(str1)

files.download('example.txt')





     |████████████████████████████████| 22.7 MB 1.4 MB/s 


Saving gandata.csv to gandata.csv
(191392,)
(63798,)
{'=', 'N', 'B', '3', '5', '4', '7', 's', '2', '(', 'c', '8', 'r', '#', '[', 'F', 'E', '+', ']', '1', 'o', '9', '6', '-', 'l', 'H', '!', 'C', 'n', 'O', 'S', ')', 'I'}
CC1Cc2c(Br)cccc2N1C(=O)Cc1nc(N2CCOCC2)cc(=O)n1C
(191392, 130, 33)
!CC1Cc2c(Br)cccc2N1C(=O)Cc1nc(N2CCOCC2)cc(=O)n1CEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 130, 33)]    0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        296960      ['input_1[0][0]']                
                                 (None, 256),                             

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 130, 33)]    0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        296960      ['input_1[0][0]']                
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                                  
 concatenate (Concatenate)      (None, 512)          0           ['lstm[0][1]',                   
                                                                  'lstm[0][2]']             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:119: RuntimeWarning: divide by zero encountered in log


CC(C)(C)C1CCC2(CC1)CN(c1ccc(F)
CC(C)(C)C1CCC2(CC1)CN(c1ccc(F)cc1)C(=O)N2Cc1ccc(C(=O)Nc2nn[nH]n2)cc1
96.0 percent wrongly formatted smiles
